In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [121]:
class VNL:
    def __init__(self):
        self.all_tables = []
        self.items = ['best-scorers', 'best-attackers','best-blockers', 'best-servers', 'best-setters', 'best-diggers', 'best-receivers']

    def vnl_requests(self, url):

        for item in self.items:
            headers = {
            'authority': 'en.volleyballworld.com',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
            'cache-control': 'max-age=0',
            'cookie': '_tt_enable_cookie=1; _ttp=mC9cwvZDEfFvm84XhcY_UpCFQS1; __qca=P0-22326168-1696070767255; _hjSessionUser_3054680=eyJpZCI6Ijc5NjFiZWRhLWFiYmItNTFmMi05YjZlLTljMWE4Mjg0Yzk2MiIsImNyZWF0ZWQiOjE2OTYwNzA3ODA1MjcsImV4aXN0aW5nIjp0cnVlfQ==; _vwo_uuid_v2=DB3B3C099E1E6AF8DAD0304C6CC4A4098|a687021c2fb118a0e3b8b34c2e8eef96; _hjSessionUser_1859763=eyJpZCI6IjcwNmIzOWRmLWU2ZWQtNWIyNC04YzgxLWQxYjczMmQxMWU2NCIsImNyZWF0ZWQiOjE2OTYwNzA3NjYxMjIsImV4aXN0aW5nIjp0cnVlfQ==; _ga=GA1.1.173820319.1696070762; _gid=GA1.2.283440947.1696070762; OptanonAlertBoxClosed=2023-09-30T22:00:03.458Z; Language=en; fivbCookies=true; SC_ANALYTICS_GLOBAL_COOKIE=acbe818e7254478b85d066a48829924e|False; _gcl_au=1.1.60668392.1704369909; _fbp=fb.1.1705345778579.1082455755; _gid=GA1.2.283440947.1696070762; _hjSession_1859763=eyJpZCI6ImU4MjU4YTllLWIwZGMtNDg5Ny1hMTE5LTEzYTUxOTgxOGYzOCIsImMiOjE3MDY2MjQzNzUwNDEsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjowLCJzcCI6MX0=; _ga=GA1.1.173820319.1696070762; _ga_R215V1S3VM=GS1.1.1706624374.16.1.1706626687.39.0.0; _ga_5ZY57D5RSS=GS1.1.1706626330.32.1.1706626688.37.0.0; OptanonConsent=isIABGlobal=false&datestamp=Tue+Jan+30+2024+11%3A58%3A08+GMT-0300+(Hor%C3%A1rio+Padr%C3%A3o+de+Bras%C3%ADlia)&version=6.18.0&hosts=&consentId=99d3274d-8d3a-4770-9823-42325ef10494&interactionCount=2&landingPath=NotLandingPage&groups=C0001%3A1%2CC0004%3A1%2CC0002%3A1%2CC0003%3A1&AwaitingReconsent=false&geolocation=BR%3BPE; _ga_0XQMFZ8Y93=GS1.2.1706624375.20.1.1706626688.0.0.0',
            'if-none-match': '"fHr8noq2CUqyI5YwyBcL9KGBMAQ"',
            'referer': f'{url}',
            'sec-ch-ua': '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'document',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-site': 'same-origin',
            'sec-fetch-user': '?1',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
            }

            response = requests.request("GET", url, headers=headers)
            content = response.text
        return content
    
    def vnl_getlinks(self, url):
        con = 0
        url_base = 'https://en.volleyballworld.com'
        url_table = ""

            
        content_pg = self.vnl_requests(url)
        source_pg = BeautifulSoup(content_pg, 'lxml')
            
        ul = source_pg.find('ul', attrs={'class':'d3-o-nav__list d3-o-list d3-o-nav__list--squared'})
        links = [url_base + i['href'] for i in ul.find_all('a')]
        for i in links:
            con+=1            
            print(f"esse {i} é o link da tabela {con}")
                           
        url_table = links
        return url_table
            
    def vnl_gettables(self, url):
        content_pg = self.vnl_requests(url)
        source_pg = BeautifulSoup(content_pg, 'lxml')
        con = 0
        while True:
            try:
                headers = [th.text for th in source_pg.find_all('th')]
                tbody = source_pg.find('tbody')
                rows = [[tr.get_text(strip=True) for tr in row] for row in tbody]

                table = [dict(zip(headers, row)) for row in rows]

                t = {'id_table':url, 'tabela':table}
                self.all_tables.append(t)
                con +=1
            except:
                print(f"got an erro in: {url}")
            break

        for table in self.all_tables:
            for row in table['tabela']:
                row['competition'] = "MEN'S VNL 2023 Finals"
                row['ano'] = "2023"
            
        return self.all_tables
    
    def vnl_json(self, table):
        name = "MEN'S_VNL_2023_FINALS.json"
        import json
        with open(name, 'w', encoding='utf-8') as json_file:
            json.dump(table, json_file, ensure_ascii=False, indent=2)    

In [131]:
vnl = VNL()

In [94]:
res = vnl.vnl_getlinks('https://en.volleyballworld.com/volleyball/competitions/volleyball-nations-league/2023/finals-statistics/men/best-diggers/')

esse https://en.volleyballworld.com/volleyball/competitions/volleyball-nations-league/2023/finals-statistics/men/best-scorers/ é o link da tabela 1
esse https://en.volleyballworld.com/volleyball/competitions/volleyball-nations-league/2023/finals-statistics/men/best-attackers/ é o link da tabela 2
esse https://en.volleyballworld.com/volleyball/competitions/volleyball-nations-league/2023/finals-statistics/men/best-blockers/ é o link da tabela 3
esse https://en.volleyballworld.com/volleyball/competitions/volleyball-nations-league/2023/finals-statistics/men/best-servers/ é o link da tabela 4
esse https://en.volleyballworld.com/volleyball/competitions/volleyball-nations-league/2023/finals-statistics/men/best-setters/ é o link da tabela 5
esse https://en.volleyballworld.com/volleyball/competitions/volleyball-nations-league/2023/finals-statistics/men/best-diggers/ é o link da tabela 6
esse https://en.volleyballworld.com/volleyball/competitions/volleyball-nations-league/2023/finals-statistics/

In [108]:
len(res)

7

In [132]:
for i in res[:]:
    vnl.vnl_gettables(i)

In [133]:
vnl.vnl_json(vnl.all_tables)